In [76]:
import requests
from massive import RESTClient
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from datetime import date

In [121]:
class Analyze:
    """
        Makes a dictionary for each day and gives RSI, price value, and number of transactions/shares 
        traded
    """

    def __init__(self, client, day_range, ticker):
        self.client = client
        self.today = datetime.today().strftime('%Y-%m-%d')
        self.day_range = day_range
        self.ticker = ticker

    """
        Converts milliseconds timestamp to more readable format
    """
    def convert_timestamp(self, ts):
        return datetime.fromtimestamp(ts/1000).strftime('%Y-%m-%d')

    """
        Gets RSI values over time frame and for a specific ticker, also 
        gets the minimum date, as this will give me the last 365 trading days
    """
    def get_rsi(self, timespan='day', adjusted=True, window=14, series_type='close', order='desc'):
        rsi_data = self.client.get_rsi(
            ticker=self.ticker,
            timespan=timespan,
            adjusted=adjusted,
            window=window,
            series_type=series_type,
            order=order,
            limit=self.day_range
        )
        self.from_date = None
        self.time_dict = {}
        for _ in rsi_data.values:
            timestamp = self.convert_timestamp(_.timestamp)
            # sets the from date to the earliest date in the RSI data
            if pd.isna(self.from_date) or (self.from_date and timestamp < self.from_date):
                self.from_date = timestamp
            self.time_dict[timestamp] = {
                'rsi': _.value
            }
    """
        Gets prices for a specific ticker over a set time
    """
    def get_price(self, multiplier=1, timespan='day'):
        # set close, as I want to add a change in close price for each day
        last_close = None

        for _ in self.client.list_aggs(
                ticker=self.ticker, 
                multiplier=multiplier, 
                timespan=timespan, 
                from_=self.from_date, 
                to=self.today
        ):
            timestamp = self.convert_timestamp(_.timestamp)
            if not pd.isna(last_close):
                change = (_.close - last_close) / last_close
                last_close = _.close
            else:
                change = None
                last_close = _.close
            if timestamp in self.time_dict:
                self.time_dict[timestamp]['open'] = _.open
                self.time_dict[timestamp]['close'] = _.close
                self.time_dict[timestamp]['daily_roi'] = change
                self.time_dict[timestamp]['volume'] = _.volume
                self.time_dict[timestamp]['high'] = _.high
                self.time_dict[timestamp]['low'] = _.low
                self.time_dict[timestamp]['transactions'] = _.transactions
            else:
                print(f"Date {timestamp} not in Price data, but used in RSI")

    def run(self):
        self.get_rsi()
        self.get_price()
        return pd.DataFrame.from_dict(self.time_dict, orient='index')

In [122]:
"""
    use api key and set up client
"""
api_key = 'hJQGYwLtVkOmM45yOBeEOg99YqOzJAz9'
client = RESTClient(api_key=api_key)

ticker = 'AAPL'

"""
    instance of class
"""
inst = Analyze(client=client, ticker='AAPL', day_range=365)
# if want to work with df
df = inst.run()
# if want to work with dict
dict_data = inst.time_dict